## Librerias

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, current_timestamp
from pyspark.sql.types import StringType
from delta.tables import DeltaTable

## Variables

In [0]:
def variables_globales() -> dict:
    return {
        "container": dbutils.secrets.get("scope-mbc", "secret-env-container"),
        "storage_account": dbutils.secrets.get("scope-mbc", "secret-env-storage-account"),
        "path_base": f"abfss://{dbutils.secrets.get('scope-mbc', 'secret-env-container')}@{dbutils.secrets.get('scope-mbc', 'secret-env-storage-account')}.dfs.core.windows.net" # path_base = f"abfss://{container}@{storage_account}.dfs.core.windows.net"
    }

## Funciones

In [0]:
def read_landing(path: str) -> DataFrame:
    path_base = variables_globales()["path_base"]

    df = spark.read.format("parquet").load(f"{path_base}/{path}")
    columns_to_cast = [col(c).cast("string").alias(c) for c in df.columns]

    return df.select(*columns_to_cast)



In [0]:
def write_bronze(df: DataFrame, tabla: str):
    path_base = variables_globales()["path_base"]
    df.write.format("delta").mode("append").saveAsTable(tabla)